In [1]:
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt
from claspy.segmentation import BinaryClaSPSegmentation
from claspy.data_loader import load_tssb_dataset, load_has_dataset
import ruptures as rpt 
from utilsCPD import *

In [2]:
df_TS_CL = load_has_dataset()

In [3]:
METRICS = {}
IDs = [10,14,7,182,225,19,185,33,36,87,88,210,11,20,23,243,247,91,95,96,100,141,91,95,245]


for id in IDs:
    print(id)
    METRICS[id]  = {}

    dataset, window_size, true_cps,labels, time_series = df_TS_CL.iloc[id,:]
    algo_c = rpt.KernelCPD(kernel="rbf", min_size=2).fit(
        time_series
    ) 
    penalty_value = 10 # beta
    cps_KCPA_PELT = algo_c.predict(pen=penalty_value)
    cps = np.array(cps_KCPA_PELT)[:-1]
    f1,AUC_run = f_measure({'0':true_cps},cps,100)
    METRICS[id]['F1'] = f1
    METRICS[id]['AUC'] = AUC_run
    METRICS[id]['Covering'] = covering({'0':true_cps},cps,time_series.shape[0])


f1,cov,AUC = [],[],[]
for k,metrics in METRICS.items():
    if len(metrics)>1:
        f1.append(metrics['F1'])
        cov.append(metrics['Covering'])
        AUC.append(metrics['AUC'])

10
14
7
182
225
19
185
33
36
87
88
210
11
20
23
243
247
91
95
96
100
141
91
95
245


In [4]:
f1,cov,AUC = [],[],[]
for k,metrics in METRICS.items():
    if len(metrics)>1:
        f1.append(metrics['F1'])
        cov.append(metrics['Covering'])
        AUC.append(metrics['AUC'])

print(np.mean(f1))
print(np.mean(cov))
print(np.mean(AUC))

0.41465747239404527
0.585138565707187
0.6571305605290371


In [5]:
Occupancy = pd.read_csv('./datasets/Occupancy/Occupancy.csv').iloc[:,1:]
algo_c = rpt.KernelCPD(kernel="rbf", min_size=2).fit(
        Occupancy.to_numpy()
    ) 
penalty_value = 50 # beta
cps_KCPA_PELT = algo_c.predict(pen=penalty_value)
cps = np.array(cps_KCPA_PELT)[:-1]
print(cps)
GroundTruth = [15,831,1455,2271,2895,3743,4271,5183,5759,6655,7215,8095]
f1,AUC_run = f_measure({'0':GroundTruth},cps,30)
print(covering({'0':GroundTruth},cps,Occupancy.shape[0]))
print(AUC_run)

[ 153  827  919 1454 1527 2269 2358 2896 2960 3806 4107 6654 6742 7035
 7213 7364 7492 8086]
0.6407256925917354
0.5182186234817814


In [6]:
import zipfile
import json

zip_file_path = './datasets/MNISTSeq.zip'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_contents = zip_ref.namelist()
    json_file_name = zip_contents[0]
    with zip_ref.open(json_file_name) as json_file:
        data = json.load(json_file)

In [10]:
F1,Covering,AUC = [],[],[]

for id, k in enumerate(data.keys()):
    ground_truths = data[k]['target']
    ts = data[k]['data']
    n = ground_truths[-1] + 200
    algo_c = rpt.KernelCPD(kernel="rbf", min_size=1).fit(
        np.array(ts)
    ) 
    penalty_value = 1# beta
    cps_KCPA_PELT = algo_c.predict(pen=penalty_value)
    cps = np.array(cps_KCPA_PELT)[:-1]
    print(cps)
    f1, AUC_score = f_measure({'0':ground_truths},cps,20)
    covering_score = covering({'0':ground_truths},cps,n)
    F1.append(f1)
    AUC.append(AUC_score)
    Covering.append(covering_score)

print(np.mean(AUC))
print(np.std(AUC))
print(np.mean(Covering))
print(np.std(Covering))

[200 400]
[  4 200 374 381 400]
[200 400]
[  4 200 287 400]
[200 400]
[200 287 400 600]
[200 400 404 600 774 781]
[ 87 200 204 400 600]
[200 287 400 600]
[  4 200 287 400 600]
[200 204 400 600 687 800]
[174 181 200 400 600 800]
[200 400 600 800]
[200 204 400 600 800]
[200 400 600 800]
0.8955555555555555
0.08489407684399873
0.9463222222222224
0.05303691819250082
